In [1]:
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [2]:
import torch
print(torch.cuda.is_available())  # Should return True if GPU is available
print(torch.cuda.device_count())  # Number of available GPUs
print(torch.cuda.get_device_name(0))  # Name of your GPU

True
1
NVIDIA GeForce RTX 3080 Ti


In [ ]:
!pip install ultralytics

In [3]:
import os
import json
import hashlib
import pandas as pd
from PIL import Image
import cv2
from tqdm import tqdm

# Base directory for your dataset
base_dir = "S:/Assignment2/Senior-Design-VIAD-4"

# Define dataset paths dynamically
test_path = os.path.join(base_dir, "test")
train_path = os.path.join(base_dir, "train")
valid_path = os.path.join(base_dir, "valid")

# Create directories for YOLOv8 formatted dataset
output_base_dir = os.path.join(base_dir, "yolov8", "data", "coco")
os.makedirs(os.path.join(output_base_dir, "labels"), exist_ok=True)
os.makedirs(os.path.join(output_base_dir, "images"), exist_ok=True)

# Sanitize and shorten file names to remove problematic characters and reduce path length
def sanitize_and_shorten_filename(filename):
    sanitized_filename = filename.replace(" ", "_").replace(",", "_").replace("(", "").replace(")", "").replace("'", "").replace("&", "")
    short_filename = hashlib.md5(sanitized_filename.encode()).hexdigest()[:10]  # 10-character hash
    return short_filename

# Function to create YOLOv8 labels from JSON annotations
def create_labels(path_in, path_out):
    json_files = [file for file in os.listdir(path_in) if file.lower().endswith('.json')]  # Handle lower case for consistency
    
    if not json_files:
        print(f"No JSON annotation files found in {path_in}. Exiting.")
        return

    # Load the JSON annotations
    with open(os.path.join(path_in, json_files[0]), 'r') as f:
        data = json.load(f)
        
    df_images = pd.DataFrame(data['images'])
    df_images.rename(columns={'id': 'image_id'}, inplace=True)
    df_categories = pd.DataFrame(data['categories'])
    df_categories.rename(columns={'id': 'category_id'}, inplace=True)
    df_annotations = pd.DataFrame(data['annotations'])
    
    merged_df = pd.merge(df_annotations, df_categories, how='left', left_on='category_id', right_on='category_id')
    final_df = pd.merge(merged_df, df_images, how='left', left_on='image_id', right_on='image_id')
    
    df = final_df[['category_id', 'bbox', 'file_name']].copy()
    df[['bbox_x', 'bbox_y', 'bbox_width', 'bbox_height']] = pd.DataFrame(df['bbox'].apply(lambda x: pd.Series(x)))

    # Drop the original 'bbox' column and duplicates
    df.drop('bbox', axis=1, inplace=True)
    df.drop_duplicates(inplace=True)

    # Create output directories for labels and images
    label_dir = os.path.join(output_base_dir, "labels", path_out)
    image_dir = os.path.join(output_base_dir, "images", path_out)
    os.makedirs(label_dir, exist_ok=True)
    os.makedirs(image_dir, exist_ok=True)

    # Keep track of missing files
    missing_files = []

    # Create label files and process images
    for file_name, group in df.groupby('file_name'):
        shortened_file_name = sanitize_and_shorten_filename(file_name)
        full_label_path = os.path.join(label_dir, shortened_file_name + ".txt")

        with open(full_label_path, 'w') as f:
            for _, row in group.iterrows():
                class_index = row['category_id']

                # Dynamically handle image size for bounding box normalization
                source_image_path = os.path.join(path_in, file_name)
                try:
                    with Image.open(source_image_path) as img:
                        width, height = img.size
                except FileNotFoundError:
                    missing_files.append(source_image_path)
                    continue
                
                x_center = (row['bbox_x'] + row['bbox_width'] / 2) / width
                y_center = (row['bbox_y'] + row['bbox_height'] / 2) / height
                width_norm = row['bbox_width'] / width
                height_norm = row['bbox_height'] / height
                f.write(f"{class_index} {x_center} {y_center} {width_norm} {height_norm}\n")

    image_files = [file for file in os.listdir(path_in) if file.lower().endswith(('.jpg', '.png'))]  # Lowercase for consistency
    
    print(f"Available image files in {path_in}: {image_files}") 
    
    # Process images with a progress bar
    for file_name in tqdm(image_files, desc=f"Processing {path_out} images"):
        source_file = os.path.join(path_in, file_name)
        
        if not os.path.exists(source_file):
            print(f"File not found: {source_file}. Skipping.")
            missing_files.append(source_file)  
            continue

        shortened_image_name = sanitize_and_shorten_filename(file_name)
        destination_file = os.path.join(image_dir, shortened_image_name + ".jpg")
        
        try:
            image = Image.open(source_file)
            if image.mode == "RGBA":
                background = Image.new("RGB", image.size, (255, 255, 255))
                background.paste(image, mask=image.split()[3])
                background.save(destination_file, "JPEG")
            else:
                image = image.convert("RGB")
                image.save(destination_file)
        except FileNotFoundError as e:
            print(f"Error: {e}. Skipping this image.")
            continue

    # Log missing files to a text file
    if missing_files:
        print("\nLogging missing files...")
        with open('missing_files.log', 'w') as log_file:
            for missing_file in missing_files:
                log_file.write(f"{missing_file}\n")
        print(f"Missing files logged in 'missing_files.log'.")

    print(f"{path_out} data and labels created successfully!\n")

# Create labels and images for train, valid, and test datasets
create_labels(train_path, 'train')
create_labels(valid_path, 'valid')
create_labels(test_path, 'test')


Available image files in S:/Assignment2/Senior-Design-VIAD-4\train: ['-0378D390-5AE2-4D09-84BB-387E850F5E5A-png_jpg.rf.4031a3364d744506724c3fb449927e17.jpg', '-0378D390-5AE2-4D09-84BB-387E850F5E5A-png_jpg.rf.558d2065150601e480aa5fefe334b407.jpg', '-0378D390-5AE2-4D09-84BB-387E850F5E5A-png_jpg.rf.7adf086300070433df16056698f87b6b.jpg', '-046E9840-F025-4749-B3AB-19402B34E739-png_jpg.rf.1f0a0a68232a44d39f2eef6badd3218f.jpg', '-046E9840-F025-4749-B3AB-19402B34E739-png_jpg.rf.26b15251d64bb01f2d3a90db6b4a329b.jpg', '-046E9840-F025-4749-B3AB-19402B34E739-png_jpg.rf.5ac7581c184e39e3d11a9f8cf55a09b7.jpg', '-046E9840-F025-4749-B3AB-19402B34E739-png_jpg.rf.8ab3060bfe975f4172616cc40049cc5d.jpg', '-046E9840-F025-4749-B3AB-19402B34E739-png_jpg.rf.92e4586a048800e03b734a03de11f843.jpg', '-046E9840-F025-4749-B3AB-19402B34E739-png_jpg.rf.b345dd82271499be42fe38163f73aeb9.jpg', '-0471C451-8CA5-4CEA-8EFD-3B0F147BF4B7-png_jpg.rf.56974aab596c98c3c503df178909fdc8.jpg', '-0471C451-8CA5-4CEA-8EFD-3B0F147BF4B7-pn

Processing train images: 100%|██████████████████████████████████████████████████| 32511/32511 [00:48<00:00, 664.57it/s]


train data and labels created successfully!

Available image files in S:/Assignment2/Senior-Design-VIAD-4\valid: ['-0B43DB53-2377-4D6D-B5B8-60CBBAB90F88-png_jpg.rf.83cb6d5841f4f1176eb75f7d750f7f34.jpg', '-0DEDC30B-FD9A-4A41-ABC0-DBEDE7C1C712-png_jpg.rf.fa6cacfe7c11a6728c95bc309dd7768a.jpg', '-1058CFB8-55AA-4270-937C-FAB55C49F17A-png_jpg.rf.79d4476cbcc937aad559188efc218a29.jpg', '-1058CFB8-55AA-4270-937C-FAB55C49F17A-png_jpg.rf.d40b6fd1034d7fbfd698ef5156f760de.jpg', '-12-HDCondellJoseMiguelInfante_2020-08-04_07h30min00s000ms-064_jpg.rf.dcd7bc32a31aee32ba08108d23d0c33d.jpg', '-12-HDCondellJoseMiguelInfante_2020-08-04_07h30min00s000ms-652_jpg.rf.50c6fadfc4f252fabdcb481f76b7b92d.jpg', '-12-HDCondellJoseMiguelInfante_2020-08-11_07h30min00s000ms-270_jpg.rf.e991d26cbd6d06a8cfdbb1b7054855b9.jpg', '-12D2734F-0757-4B08-9819-D3704009B312-png_jpg.rf.c575a163797696ead38ade2c01d77dd5.jpg', '-192C4A31-07E9-4438-9D0D-63CF545747C8-png_jpg.rf.8dcfa7166594b451ccc3d7ce5b984e27.jpg', '-1D5949D1-5187-454E-8

Processing valid images: 100%|████████████████████████████████████████████████████| 4246/4246 [00:06<00:00, 686.88it/s]


valid data and labels created successfully!

Available image files in S:/Assignment2/Senior-Design-VIAD-4\test: ['-0573F94C-289D-4503-A571-94C6C8C5674C-png_jpg.rf.203ce892cfb5f1007edfffc3583c559c.jpg', '-13AD8372-CDDB-4DCF-B8F9-DC10D210064B-png_jpg.rf.e8217e5555bb565dc324989173877f5a.jpg', '-1F999A49-2799-4FBC-811D-D10A165F8B02-png_jpg.rf.8fc376c8a650e3c8839e8342a3f26c00.jpg', '-2-_jpeg_jpg.rf.c1f6abdd0cff3a6bf353478bc921328b.jpg', '-2526BBBE-6B96-49DC-94B9-880E133FA549-png_jpg.rf.aa9a18ed7c7033b8b15e8b819ad92558.jpg', '-2B9B8378-CF94-4121-9F57-AA504E361DEC-png_jpg.rf.179e73b0674f27afb3efae0f7e99c7e2.jpg', '-2B9B8378-CF94-4121-9F57-AA504E361DEC-png_jpg.rf.b3e0a68cd72a91d93ed767eced1e5947.jpg', '-3130BF7E-8143-44C3-AC0D-8068C9108560-png_jpg.rf.e43cab7a151c3189f220dd5bd3f8d5d1.jpg', '-341F28F4-B515-475F-8729-F40EFDDDF0C2-png_jpg.rf.53a8aa8b0a896d7eeb3c968959f9e99f.jpg', '-3B056DBB-3097-4BCB-A38A-F66FA7470A6C-png_jpg.rf.46d23b25766e9b1676216e03e9a88cdf.jpg', '-3B056DBB-3097-4BCB-A38A-F66F

Processing test images: 100%|█████████████████████████████████████████████████████| 2165/2165 [00:03<00:00, 642.83it/s]

test data and labels created successfully!



In [4]:
# Missing file cleanup code
def remove_missing_labels(missing_images, label_dir):
    for image_name in missing_images:
        label_name = image_name.replace(".jpg", ".txt").replace(".png", ".txt")
        label_path = os.path.join(label_dir, label_name)
        if os.path.exists(label_path):
            print(f"Deleting label file: {label_path}")
            os.remove(label_path)
        else:
            print(f"Label file not found for: {label_path}")
    print("Label cleanup completed.")

# List of missing images from the dataset
missing_images = [
    "outdoor-vertical-traffic-light-green-color-outdoor-vertical-traffic-light-blue-sky-trees-around-traffic-control-concept-221609452_jpg.rf.b97b8c4cc1c6771f0e1fe2205e60b112.jpg",
    "outdoor-vertical-traffic-light-green-color-outdoor-vertical-traffic-light-blue-sky-trees-around-traffic-control-concept-221609452_jpg.rf.c1679bfa71e67fc601ce86a76a437348.jpg",
    "outdoor-vertical-traffic-light-green-color-outdoor-vertical-traffic-light-blue-sky-trees-around-traffic-control-concept-221609452_jpg.rf.f984d1f93297c1a354c138bedc38a169.jpg"
]

remove_missing_labels(missing_images, "S:/Assignment2/Senior-Design-VIAD-4/labels/train")

Label file not found for: S:/Assignment2/Senior-Design-VIAD-4/labels/train\outdoor-vertical-traffic-light-green-color-outdoor-vertical-traffic-light-blue-sky-trees-around-traffic-control-concept-221609452_jpg.rf.b97b8c4cc1c6771f0e1fe2205e60b112.txt
Label file not found for: S:/Assignment2/Senior-Design-VIAD-4/labels/train\outdoor-vertical-traffic-light-green-color-outdoor-vertical-traffic-light-blue-sky-trees-around-traffic-control-concept-221609452_jpg.rf.c1679bfa71e67fc601ce86a76a437348.txt
Label file not found for: S:/Assignment2/Senior-Design-VIAD-4/labels/train\outdoor-vertical-traffic-light-green-color-outdoor-vertical-traffic-light-blue-sky-trees-around-traffic-control-concept-221609452_jpg.rf.f984d1f93297c1a354c138bedc38a169.txt
Label cleanup completed.


In [4]:
# YOLOv8 Dataset Creation and Model Training
import os
import yaml

# Define the base directory for the output
output_base_dir = "S:/Assignment2/Senior-Design-VIAD-4/yolov8/data/coco"

# Paths to train and validation images and labels directories
train_images_dir = os.path.join(output_base_dir, "images/train")
val_images_dir = os.path.join(output_base_dir, "images/valid")
train_labels_dir = os.path.join(output_base_dir, "labels/train")
val_labels_dir = os.path.join(output_base_dir, "labels/valid")

# Number of classes (This should match the length of class_names)
num_classes = 37

# List of class names
class_names = ['cars-bikes-people', 'Bus', 'Bushes', 'Person', 'Truck', 'backpack', 'bench',
               'bicycle', 'boat', 'branch', 'car', 'chair', 'clock', 'crosswalk', 'door',
               'elevator', 'fire_hydrant', 'green_light', 'gun', 'handbag', 'motorcycle',
               'person', 'pothole', 'rat', 'red_light', 'scooter', 'sheep', 'stairs', 'stop_sign',
               'suitcase', 'traffic_light', 'traffic_cone', 'train', 'tree', 'truck', 'umbrella',
               'yellow_light']

# Create the dataset.yaml information
dataset_info = {
    'train': train_images_dir,  
    'val': val_images_dir,      
    'nc': num_classes,         
    'names': class_names       
}

# Create and save the dataset.yaml file
dataset_yaml_path = os.path.join(output_base_dir, "dataset.yaml")
with open(dataset_yaml_path, 'w') as file:
    yaml.dump(dataset_info, file, default_flow_style=False)

print(f"dataset.yaml file has been created at: {dataset_yaml_path}")

# Function to clean label files and remove invalid class indices
def clean_labels(labels_dir):
    max_class_index = num_classes - 1  # Maximum valid class index (0 to num_classes-1)
    
    for label_file in os.listdir(labels_dir):
        if label_file.endswith(".txt"):
            label_path = os.path.join(labels_dir, label_file)
            with open(label_path, 'r') as f:
                lines = f.readlines()
            
            # Filter out invalid class labels
            valid_lines = []
            for line in lines:
                class_index = int(line.split()[0])  # Get class index from the label line
                if class_index <= max_class_index:
                    valid_lines.append(line)
                else:
                    print(f"Invalid class index {class_index} in file {label_file}, removing it.")
            
            # Write back only the valid labels
            with open(label_path, 'w') as f:
                f.writelines(valid_lines)

    print(f"Invalid class labels check completed for {labels_dir}.")

# Clean invalid labels in both train and validation datasets
clean_labels(train_labels_dir)
clean_labels(val_labels_dir)

print("Invalid class labels cleaning process completed for both train and validation datasets.")


dataset.yaml file has been created at: S:/Assignment2/Senior-Design-VIAD-4/yolov8/data/coco\dataset.yaml
Invalid class labels check completed for S:/Assignment2/Senior-Design-VIAD-4/yolov8/data/coco\labels/train.
Invalid class labels check completed for S:/Assignment2/Senior-Design-VIAD-4/yolov8/data/coco\labels/valid.
Invalid class labels cleaning process completed for both train and validation datasets.


In [7]:
from ultralytics import YOLO

def train_yolo_model():
    print("Initializing YOLOv8 model for training...")

    # Initialize the YOLOv8 model (pre-trained on COCO)
    model = YOLO('yolov8n.pt')  # You can also use 'yolov8s.pt' or 'yolov8m.pt' for larger models

    print("Starting training...")
    # Ensure the correct path to dataset.yaml
    dataset_yaml_path = "S:/Assignment2/Senior-Design-VIAD-4/yolov8/data/coco/dataset.yaml"

    # Train the model
    results = model.train(data=dataset_yaml_path, epochs=20, imgsz=320, device='gpu')

    print("Training complete.")
    
    # Save the best model after training
    model.save('best.pt')  
    print("Model saved as 'best.pt'.")

train_yolo_model()





Initializing YOLOv8 model for training...


100%|█████████████████████████████████████████████████████████████████████████████| 6.25M/6.25M [00:02<00:00, 3.23MB/s]


Starting training...
Ultralytics 8.3.13  Python-3.10.11 torch-2.4.1+cu118 CUDA:gpu (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=S:/Assignment2/Senior-Design-VIAD-4/yolov8/data/coco/dataset.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=gpu, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, 

100%|█████████████████████████████████████████████████████████████████████████████| 5.35M/5.35M [00:01<00:00, 3.44MB/s]


AMP: checks passed 


...

                   all       4246       7242       0.61      0.451      0.475      0.325

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      20/20     0.768G     0.9706     0.7617      1.126         23        320: 100%|██████████| 2032/2032 [03:26<00:00,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 133/133 [00:


                   all       4246       7242      0.628      0.444      0.478      0.327

20 epochs completed in 1.266 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 5.6MB
Optimizer stripped from runs\detect\train\weights\best.pt, 5.6MB

Validating runs\detect\train\weights\best.pt...
Ultralytics 8.3.13  Python-3.10.11 torch-2.4.1+cu118 CUDA:gpu (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
Model summary (fused): 186 layers, 2,691,583 parameters, 0 gradients, 6.9 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 133/133 [00:


                   all       4246       7242      0.628      0.444      0.478      0.327
                   Bus        203        275      0.772      0.684      0.744      0.593
                Bushes        207        437      0.796      0.591      0.621      0.438
                Person        191       1023      0.539     0.0811      0.164     0.0667
                 Truck         37         45      0.795      0.356      0.431      0.343
              backpack          3          3          0          0          0          0
                 bench          2          3          1          0          0          0
               bicycle        290        358      0.697      0.565      0.649      0.381
                  boat          2          4          0          0          0          0
                branch         21         62      0.494      0.331      0.353      0.207
                   car        228        463      0.761      0.579      0.639      0.449
                 chai

In [5]:
from ultralytics import YOLO

def train_yolo_model():
    print("Initializing YOLOv8 model for training...")

    # Initialize the YOLOv8 model (pre-trained on COCO)
    model = YOLO('yolov8n.pt')  # You can use 'yolov8n.pt' (nano) or other versions like 'yolov8s.pt'

    print("Starting training...")

    # Ensure the correct path to dataset.yaml
    dataset_yaml_path = "S:/Assignment2/Senior-Design-VIAD-4/yolov8/data/coco/dataset.yaml"

    # Train the model
    results = model.train(data=dataset_yaml_path, epochs=50, imgsz=300, device='gpu')

    print("Training complete.")

    # Save the best model after training
    model.save('trainModel_epoc50.pt')  
    print("Model saved as 'trainModel_epoc50.pt'.")

train_yolo_model()


Initializing YOLOv8 model for training...
Starting training...
Ultralytics 8.3.13  Python-3.10.11 torch-2.4.1+cu118 CUDA:gpu (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=S:/Assignment2/Senior-Design-VIAD-4/yolov8/data/coco/dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=300, save=True, save_period=-1, cache=False, device=gpu, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show

train: Scanning S:\Assignment2\Senior-Design-VIAD-4\yolov8\data\coco\labels\train.cache... 31944 images, 567 background
val: Scanning S:\Assignment2\Senior-Design-VIAD-4\yolov8\data\coco\labels\valid.cache... 4189 images, 57 backgrounds, 0


Plotting labels to runs\detect\train6\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 63 weight(decay=0.0), 70 weight(decay=0.0005), 69 bias(decay=0.0)
Image sizes 320 train, 320 val
Using 8 dataloader workers
Logging results to runs\detect\train6
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50     0.803G      1.455       3.75      1.444         30        320: 100%|██████████| 2032/2032 [03:51<00:00,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 133/133 [00:


                   all       4246       7242      0.658      0.252      0.241      0.142

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       4246       7242      0.617      0.461      0.492      0.341

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      50/50     0.782G     0.8742     0.6081      1.057         17        320: 100%|██████████| 2032/2032 [03:16<00:00,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 133/133 [00:


                   all       4246       7242      0.616      0.462      0.493      0.341

50 epochs completed in 3.095 hours.
Optimizer stripped from runs\detect\train6\weights\last.pt, 5.6MB
Optimizer stripped from runs\detect\train6\weights\best.pt, 5.6MB

Validating runs\detect\train6\weights\best.pt...
Ultralytics 8.3.13  Python-3.10.11 torch-2.4.1+cu118 CUDA:gpu (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
Model summary (fused): 186 layers, 2,691,583 parameters, 0 gradients, 6.9 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 133/133 [00:


                   all       4246       7242       0.62       0.46      0.493      0.341
                   Bus        203        275      0.801      0.747      0.792       0.65
                Bushes        207        437      0.821      0.663      0.656      0.474
                Person        191       1023      0.478      0.101      0.147     0.0636
                 Truck         37         45      0.699      0.333      0.418      0.324
              backpack          3          3          0          0          0          0
                 bench          2          3          1          0          0          0
               bicycle        290        358      0.655      0.603      0.646      0.393
                  boat          2          4          0          0          0          0
                branch         21         62       0.57      0.363      0.468      0.267
                   car        228        463      0.765        0.6      0.665      0.477
                 chai

In [9]:
import matplotlib.pyplot as plt
import torch
from ultralytics import YOLO
import os
import cv2

# Load the best-trained model
model = YOLO('best.pt')  # Load your best model

# 1. **Validate the Model on the Validation Set**
print("Validating the model on validation set...")
validation_results = model.val(data="S:/Assignment2/Senior-Design-VIAD-4/yolov8/data/coco/dataset.yaml", imgsz=320)
# Ensure the dataset.yaml path is correct. This will perform validation and print metrics.

# Print validation metrics (optional)
print("Validation completed! Here are the key validation results:")

# **Test the Model on a Specific Test Image (from the test set)**
test_image_path = 'S:/Assignment2/Senior-Design-VIAD-4/test'  # Replace with an actual test image path
print(f"Running inference on test image: {test_image_path}")

# Perform inference on the test image
test_results = model.predict(source=test_image_path)

# Check if the inference results contain valid detections for the test image
if test_results and len(test_results[0].boxes) > 0:
    print(f"Objects detected in the test image: {len(test_results[0].boxes)}")
    
    # Display the image with detection results
    print("Displaying detection results for the test image...")
    plt.imshow(test_results[0].plot())  # Plot the detection results on the image
    plt.axis('off')  # Remove axis labels for a cleaner image
    plt.show()  # Display the image
else:
    print("No objects detected in the test image.")



Validating the model on validation set...
Ultralytics 8.3.13  Python-3.10.11 torch-2.4.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
Model summary (fused): 186 layers, 2,691,583 parameters, 0 gradients, 6.9 GFLOPs


val: Scanning S:\Assignment2\Senior-Design-VIAD-4\yolov8\data\coco\labels\valid.cache... 4189 images, 57 backgrounds, 0
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 266/266 [00:


                   all       4246       7242      0.627      0.444      0.478      0.327
                   Bus        203        275      0.773      0.684      0.744      0.594
                Bushes        207        437      0.795       0.59      0.621      0.438
                Person        191       1023      0.541     0.0811      0.163     0.0663
                 Truck         37         45      0.795      0.356      0.431      0.343
              backpack          3          3          0          0          0          0
                 bench          2          3          1          0          0          0
               bicycle        290        358      0.694      0.565       0.65       0.38
                  boat          2          4          0          0          0          0
                branch         21         62      0.483      0.331       0.35      0.206
                   car        228        463      0.755      0.577      0.636      0.448
                 chai

<Figure size 640x480 with 1 Axes>

In [11]:
# Load the best-trained model
model = YOLO('trainModel_epoc50.pt')  # Load your best model

# 1. **Validate the Model on the Validation Set**
print("Validating the model on validation set...")
validation_results = model.val(data="S:/Assignment2/Senior-Design-VIAD-4/yolov8/data/coco/dataset.yaml", imgsz=320)
# Ensure the dataset.yaml path is correct. This will perform validation and print metrics.

# Print validation metrics (optional)
print("Validation completed! Here are the key validation results:")

#print(f"Validation mAP@0.5:0.95: {validation_results['metrics/mAP50-95']:.4f}")

# 2. **Test the Model on a Specific Test Image (from the test set)**
test_image_path = 'S:/Assignment2/Senior-Design-VIAD-4/test'  # Replace with an actual test image path
print(f"Running inference on test image: {test_image_path}")

# Perform inference on the test image
test_results = model.predict(source=test_image_path)

# Check if the inference results contain valid detections for the test image
if test_results and len(test_results[0].boxes) > 0:
    print(f"Objects detected in the test image: {len(test_results[0].boxes)}")
    
    # Display the image with detection results
    print("Displaying detection results for the test image...")
    plt.imshow(test_results[0].plot())  # Plot the detection results on the image
    plt.axis('off')  # Remove axis labels for a cleaner image
    plt.show()  # Display the image
else:
    print("No objects detected in the test image.")

Validating the model on validation set...
Ultralytics 8.3.13  Python-3.10.11 torch-2.4.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
Model summary (fused): 186 layers, 2,691,583 parameters, 0 gradients, 6.9 GFLOPs


val: Scanning S:\Assignment2\Senior-Design-VIAD-4\yolov8\data\coco\labels\valid.cache... 4189 images, 57 backgrounds, 0
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 266/266 [00:


                   all       4246       7242      0.616      0.463      0.493      0.341
                   Bus        203        275        0.8      0.749      0.792      0.651
                Bushes        207        437      0.815      0.661      0.654      0.473
                Person        191       1023      0.462      0.101      0.147     0.0634
                 Truck         37         45      0.696      0.333      0.418      0.321
              backpack          3          3          0          0          0          0
                 bench          2          3          1          0          0          0
               bicycle        290        358      0.646      0.606      0.645      0.392
                  boat          2          4          0          0          0          0
                branch         21         62      0.558      0.387      0.467      0.269
                   car        228        463       0.75      0.611      0.665      0.477
                 chai

<Figure size 640x480 with 1 Axes>

In [12]:
# **Test the Model on a Specific Test Image (from the test set)**
test_image_path = 'S:/Assignment2/Senior-Design-VIAD-4/test'  # Replace with an actual test image path
print(f"Running inference on test image: {test_image_path}")

# Perform inference on the test image
test_results = model.predict(source=test_image_path)

# Check if the inference results contain valid detections for the test image
if test_results and len(test_results[0].boxes) > 0:
    print(f"Objects detected in the test image: {len(test_results[0].boxes)}")
    
    # Display the image with detection results
    print("Displaying detection results for the test image...")
    plt.imshow(test_results[0].plot())  # Plot the detection results on the image
    plt.axis('off')  # Remove axis labels for a cleaner image
    plt.show()  # Display the image
else:
    print("No objects detected in the test image.")


Running inference on test image: S:/Assignment2/Senior-Design-VIAD-4/test


WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/2165 S:\Assignment2\Senior-Design-VIAD-4\test\-0573F94C-289D-4503-A571-94C6C8C5674C-png_jpg.rf.203ce892cfb5f1007edfffc3583c559c.jpg: 640x640 3 cars, 63.5ms
image 2/2165 S:\Assignment2\Senior-Design-VIAD-4\test\-13AD8372-CDDB-4DCF-B8F9-DC10D210064B-png_jpg.rf.e8217e5555bb565dc324989173877f5a.jpg: 640x640 1 car, 58.2ms
image 3/2165 S:\Assignment2\Senior-Design-VIAD-4\test\-1F

<Figure size 640x480 with 1 Axes>

In [13]:

# 3. **Prediction on a Random Image (not part of the validation or test set)**
random_image_path = 'S:/Assignment2/Image01.jpg'  # Replace this with the path to your random image
print(f"Running inference on random image: {random_image_path}")

# Perform inference on the random image
random_results = model.predict(source=random_image_path)

# Check if the inference results contain valid detections for the random image
if random_results and len(random_results[0].boxes) > 0:
    print(f"Objects detected in the random image: {len(random_results[0].boxes)}")
    
    # Display the image with detection results
    print("Displaying detection results for the random image...")
    plt.imshow(random_results[0].plot())  # Plot the detection results on the image
    plt.axis('off')  # Remove axis labels for a cleaner image
    plt.show()  # Display the image
else:
    print("No objects detected in the random image.")

Running inference on random image: S:/Assignment2/Image01.jpg

image 1/1 S:\Assignment2\Image01.jpg: 480x640 2 Buss, 1 Person, 58.9ms
Speed: 3.0ms preprocess, 58.9ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)
Objects detected in the random image: 3
Displaying detection results for the random image...


<Figure size 640x480 with 1 Axes>

In [25]:
import cv2
from ultralytics import YOLO

def detect_and_save(image_path, save_path):
    # Load the YOLO model
    model = YOLO('S:/Assignment2/trainModel_epoc50.pt')  # Load your trained model

    # Perform inference on the image
    results = model(image_path, conf=0.6, iou=0.5)

    # Print out detected objects to verify
    print(results[0].boxes)

    # Get the image with bounding boxes (if detected)
    result_image = results[0].plot()

    # Convert to BGR format to save using OpenCV
    result_image_bgr = cv2.cvtColor(result_image, cv2.COLOR_RGB2BGR)

    # Save the image with bounding boxes
    cv2.imwrite(save_path, result_image_bgr)
    print(f"Image saved with bounding boxes at: {save_path}")

# Path to the test image
image_path = "S:/Assignment2/Image01.jpg"
# Path to save the result image
save_path = "S:/Assignment2/Image01_with_detections.jpg"

# Call the function to detect objects and save the image
detect_and_save(image_path, save_path)



image 1/1 S:\Assignment2\Image01.jpg: 480x640 2 Buss, 1 Person, 11.6ms
Speed: 2.0ms preprocess, 11.6ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([1., 3., 1.], device='cuda:0')
conf: tensor([0.7937, 0.7248, 0.6857], device='cuda:0')
data: tensor([[1.1832e+03, 3.5373e+02, 3.1909e+03, 2.4054e+03, 7.9366e-01, 1.0000e+00],
        [3.2868e+03, 1.1401e+03, 3.5341e+03, 1.9293e+03, 7.2479e-01, 3.0000e+00],
        [4.6248e+02, 3.6219e+02, 2.4110e+03, 2.4066e+03, 6.8568e-01, 1.0000e+00]], device='cuda:0')
id: None
is_track: False
orig_shape: (3000, 4000)
shape: torch.Size([3, 6])
xywh: tensor([[2187.0457, 1379.5403, 2007.6470, 2051.6294],
        [3410.4409, 1534.7358,  247.2764,  789.2064],
        [1436.7299, 1384.4011, 1948.4978, 2044.4180]], device='cuda:0')
xywhn: tensor([[0.5468, 0.4598, 0.5019, 0.6839],
        [0.8526, 0.5116, 0.0618, 0.2631],
        [0.3592, 0.4615, 0.4871, 0.6815]], device='